<a href="https://colab.research.google.com/github/Latamila/Scikit-learn_estudos_casos/blob/main/Vendas_da_loja_Rossman.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DESCRIÇÃO
---
Rossmann opera mais de 3.000 drogarias em 7 países europeus. Atualmente, os gerentes das lojas Rossmann têm a tarefa de prever suas vendas diárias com até seis semanas de antecedência. As vendas nas lojas são influenciadas por muitos fatores, incluindo promoções, concorrência, feriados escolares e estaduais, sazonalidade e localidade. Com milhares de gestores individuais prevendo vendas com base em circunstâncias específicas, a precisão dos resultados pode ser bastante variada.

Em sua primeira competição Kaggle, Rossmann desafia você a prever 6 semanas de vendas diárias para 1.115 lojas localizadas em toda a Alemanha. Previsões de vendas confiáveis ​​permitem que os gerentes de loja criem cronogramas de equipe eficazes que aumentam a produtividade e a motivação. Ao ajudar a Rossmann a criar um modelo de previsão robusto, você ajudará os gerentes de loja a manterem o foco no que é mais importante para eles: seus clientes e suas equipes!



- Id - um Id que representa um duple (Loja, Data) dentro do conjunto de teste
- Loja – um ID exclusivo para cada loja
Vendas - o faturamento de um determinado dia (é isso que você está prevendo)
- Clientes - o número de clientes em um determinado dia
- Aberta - um indicador para saber se a loja estava aberta: 0 = fechada, 1 = aberta
- StateHoliday – indica feriado estadual. Normalmente todas as lojas, com poucas exceções, fecham nos feriados estaduais. Observe que todas as escolas estão fechadas nos feriados e fins de semana. a = feriado, b = feriado de Páscoa, c = Natal, 0 = Nenhum
- SchoolHoliday - indica se a (Loja, Data) foi afetada pelo fechamento de escolas públicas
- StoreType  - diferencia entre 4 modelos de loja diferentes: a, b, c, d
Sortimento - descreve um nível de sortimento: a = básico, b = extra, c = estendido
- CompetitionDistance - distância em metros até a loja concorrente mais próxima
- CompetitionOpenSince[Month/Year] - fornece o ano e mês aproximados da hora em que o concorrente mais próximo foi aberto
- Promoção - indica se uma loja está realizando uma promoção naquele dia
- Promo2 - Promo2 é uma promoção contínua e consecutiva para algumas lojas: 0 = loja não participa, 1 = loja participa
- Promo2Since[Ano/Semana] - descreve o ano e a semana do calendário em que a loja começou a participar da Promo2
- PromoInterval - descreve os intervalos consecutivos de início da Promo2, nomeando os meses em que a promoção é reiniciada. Por exemplo, "fevereiro, maio, agosto, novembro" significa que cada rodada começa em fevereiro, maio, agosto e novembro de qualquer ano para aquela loja

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
uploaded = files.upload()

In [ ]:
uploaded = files.upload()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
import numpy as np
import pandas as pd
df_store = pd.read_csv('store.csv')

In [ ]:
print("Shape of the Dataset:",df_store.shape)

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
print("Shape of the Dataset:",df.shape)

In [ ]:
df.head(5)

In [ ]:
df_store.head(5)

In [ ]:
df_new = df.merge(df_store, on=['Store'], how='inner')
print(df_new.shape)

- numero distinto de lojas unicas
- numero unicos de dias
- media de vendas diarias de todas as lojas

In [ ]:
print("Distinct number of Stores :", len(df_new["Store"].unique()))
print("Distinct number of Days :", len(df_new["Date"].unique()))
print("Average daily sales of all stores : ",round(df_new["Sales"].mean(),2))

In [ ]:
df_new.dtypes #tipos de dados

In [ ]:
df_new["DayOfWeek"].value_counts()

#ENGENHARIA DE ATRIBUTOS
---

In [ ]:
df_new['Date'] = pd.to_datetime(df_new['Date'], infer_datetime_format=True)
df_new['Month'] = df_new['Date'].dt.month
df_new['Quarter'] = df_new['Date'].dt.quarter
df_new['Year'] = df_new['Date'].dt.year
df_new['Day'] = df_new['Date'].dt.day
df_new['Week'] = df_new['Date'].dt.week


In [ ]:
df_new.head(5)

In [ ]:
df_new['Season'] = np.where(df_new['Month'].isin([3,4,5]),
                            'Spring',
                            np.where(df_new['Month'].isin([6,7,8]),
                                     'Summer',
                                     np.where(df_new['Month'].isin([9,10,11]),
                                              'Fall',
                                              np.where(df_new['Month'].isin([12,1,2]),
                                                       'Winter', 'None'))))

Usando o comando `head` para ver apenas os dados e as features feitas na engenharia de atributos

In [ ]:
print(df_new[['Date','Year','Month','Day','Week',
              'Quarter','Season']].head())

#VISUALIZAÇÃO DOS DADOS
---



In [ ]:
#Create a histogram to study the Daily Sales for the stores
plt.figure(figsize=(10,5))
plt.grid()
plt.hist(df_new["Sales"])
plt.title("Histogram for Store Sales")
plt.xlabel("bins")
plt.xlabel("Frequency")
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
plt.grid()
sns.kdeplot(df_new["Sales"], fill=True)
plt.title("Histogram for Store Sales")
plt.xlabel("bins")
plt.xlabel("Frequency")
plt.show()

#Explorando Colunas Numericas
---


In [ ]:
df_new.hist(figsize=(20,10))
plt.show()

#Dados faltantes
---

In [ ]:
for i in df_new.columns:
    print('Valores faltantes em', i, 'é -', ' ', df_new[i].isnull().sum()/df_new.shape[0]*100)

#Imputação com Mode Values
---

In [ ]:
df_new['CompetitionDistance'] = df_new['CompetitionDistance'].fillna(df_new['CompetitionDistance'].mode()[0])

#Checagem dupla se ainda vemos valores nulos para a coluna
---

In [ ]:
df_new["CompetitionDistance"].isnull().sum()/df_new.shape[0] * 100

#Entendendo as variáveis categóricas
---

In [ ]:
#Create the bar plot for Average Sales across different Seasons
ax = sns.barplot(x="Season", y="Sales", data=df_new)

In [ ]:
#Create the bar plot for Average Sales across different Assortments
ax = sns.barplot(x="Assortment", y="Sales", data=df_new)

In [ ]:
#Create the bar plot for Average Sales across different Store Types
ax = sns.barplot(x="StoreType", y="Sales", data=df_new)

In [ ]:
ax = sns.barplot(x="Season", y="Sales", data=df_new,estimator=np.size)

In [ ]:
ax = sns.barplot(x="Assortment", y="Sales", data=df_new,estimator=np.size)

In [ ]:
ax = sns.barplot(x="StoreType", y="Sales", data=df_new,estimator=np.size)

#ENGENHARIA DE DADOS
---
Defina uma variável para cada tipo de feature(caracteristica)

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [ ]:
target = ['Sales']

In [ ]:
numeric_columns = ["Customers","Open","Promo","Promo2","StateHoliday","SchoolHoliday",
                   "CompetitionDistance"]
categorical_columns = ["DayOfWeek","Quarter","Month","Year",
                       "StoreType","Assortment","Season"]

In [ ]:
#Defina uma função que irá consumir o dataframe bruto
def create_ohe(df, col):
    le = LabelEncoder()
    a=le.fit_transform(df_new[col]).reshape(-1,1)
    ohe = OneHotEncoder(sparse=False)
    column_names = [col+ "_"+ str(i) for i in le.classes_]
    return(pd.DataFrame(ohe.fit_transform(a),columns =column_names))

In [ ]:
#Como a função acima converte a coluna, uma de cada vez
#Criamos um loop para criar o conjunto de dados final com todos os recursos
temp = df_new[numeric_columns]
for column in categorical_columns:
    temp_df = create_ohe(df_new, column)
    temp = pd.concat([temp, temp_df], axis=1)

In [ ]:
print("Shape of Data:",temp.shape)
print("Distinct Datatypes:",temp.dtypes.unique())

In [ ]:
print(temp.columns[temp.dtypes=="object"])

In [ ]:
temp["StateHoliday"].unique()

In [ ]:
temp["StateHoliday"]= np.where(temp["StateHoliday"]== '0',0,1)
#One last check of the data type
temp.dtypes.unique()

In [ ]:
temp['StateHoliday']

#Train-test
---

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(temp,df_new[target],test_size=0.2,random_state=2018)

In [ ]:
# Divida ainda mais o conjunto de dados de treinamento em conjunto de dados de treinamento e validação com uma divisão 90:10
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train,test_size=0.1,random_state=2018)

In [ ]:
#Check the sizes of all newly created datasets
print("Shape of x_train:",x_train.shape)
print("Shape of x_val:",x_val.shape)
print("Shape of x_test:",x_test.shape)
print("Shape of y_train:",y_train.shape)
print("Shape of y_val:",y_val.shape)
print("Shape of y_test:",y_test.shape)

#Definição do desempenho da linha de base do modelo
---

In [ ]:
#CALCULA a média do conjunto de treino
mean_sales = y_train.mean()
print('Média de Vendas: ', mean_sales)

In [ ]:
#Calcula o erro medio absoluto do conjunto de teste
print('MAE para Dados De Teste: ', abs(y_test - mean_sales).mean()[0])

#Projetando a DNN
---
- Comece com arquiteturas pequenas.
- Quando arquiteturas pequenas (com duas camadas) falharem, aumente o tamanho.
- Quando redes maiores com duas camadas falharem, vá mais fundo.
- Quando redes maiores e mais profundas também falharem, vá ainda maior e mais fundo.
- Quando tudo falhar, revise os dados

#Create Deep Neural Network Architecture
---

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
model = Sequential()
model.add(Dense(150, input_dim = 44, activation='relu'))

In [ ]:
#O input_dim =44, já que a largura dos dados de treinamento=44 (consulte a seção de engenharia de dados)
model.add(Dense(1, activation = 'linear'))

In [ ]:
#configurar o modelo
model.compile(optimizer='adam', loss='mean_absolute_error',metrics=['mean_absolute_error'])

In [ ]:
#treinar o modelo
model.fit(x_train.values, y_train.values, validation_data=(x_val,y_val),epochs=10,batch_size=64)


#Testando a performance do modelo
---

In [ ]:
#Use o método de avaliação do modelo para prever
#e avaliar os conjuntos de dados de teste
result = model.evaluate(x_test.values, y_test.values)

#printa os resultados
for i in range(len(model.metrics_names)):
    print('Metric', model.metrics_names[i],':', str(round(result[i], 2)))

#Melhorando o modelo
---
Na rede a seguir, adicionamos mais duas camadas com números semelhantes de neurônios. Atualizaremos nossa função de perda para erro quadrático médio em vez de MAE

In [ ]:
model = Sequential()
model.add(Dense(150, input_dim=44, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(150, activation='relu'))
model.add(Dense(1, activation='linear'))
model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])
history = model.fit(x_train, y_train, validation_data=(x_val,y_val),epochs=10, batch_size=64)
#result = model.evaluate(x_test, y_test)

for i in range(len(model.metrics_names)):
    print('Metrica ', model.metrics_names[i],':', str(round(result[i], 2)))

In [ ]:
model = Sequential()
model.add(Dense(150,input_dim = 44,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(150,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])
model.fit(x_train,y_train, validation_data=(x_val,y_val),
epochs=15,batch_size=64)
result = model.evaluate(x_test,y_test)
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result[i],2)))

#Aumentando o número de neurônios

- duas camadas ocultas com 350 neurônios cada e usam uma configuração de modelo

- 15 epocas

In [ ]:
model = Sequential()
model.add(Dense(350,input_dim = 44,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])
model.fit(x_train,y_train, validation_data=(x_val,y_val),
epochs=15,batch_size=64)
result = model.evaluate(x_test,y_test)
for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result[i],2)))

Epoch 1/15
11444/11444 [==============================] - 43s 4ms/step - loss: 1674571.5000 - mean_absolute_error: 845.3023 - val_loss: 1203456.3750 - val_mean_absolute_error: 739.9495
Epoch 2/15
11444/11444 [==============================] - 38s 3ms/step - loss: 1162637.2500 - mean_absolute_error: 727.5677 - val_loss: 1081292.6250 - val_mean_absolute_error: 694.7469
Epoch 3/15
11444/11444 [==============================] - 38s 3ms/step - loss: 1090188.7500 - mean_absolute_error: 702.7058 - val_loss: 1016140.6875 - val_mean_absolute_error: 674.0535
Epoch 4/15
11444/11444 [==============================] - 38s 3ms/step - loss: 1050317.2500 - mean_absolute_error: 688.6359 - val_loss: 1127280.5000 - val_mean_absolute_error: 741.4191
Epoch 5/15
11444/11444 [==============================] - 38s 3ms/step - loss: 1024984.7500 - mean_absolute_error: 680.3226 - val_loss: 1001159.6250 - val_mean_absolute_error: 680.5441
Epoch 6/15
11444/11444 [==============================] - 37s 3ms/step - lo

#Conclusão

Podemos ver algumas melhorias quando usamos uma arquitetura que foi construída com um número maior de neurônios. Podemos usar o histórico, pós-treinamento, para visualizar e entender a curva de aprendizado do modelo.

In [57]:
from keras.callbacks import History
history = History()
model = Sequential()
model.add(Dense(350,input_dim = 44,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(350,activation="relu"))
model.add(Dense(1,activation = "linear"))
model.compile(optimizer='adam',loss="mean_squared_error",metrics=["mean_absolute_error"])
model.fit(x_train,y_train, validation_data=(x_val,y_val),
epochs=15,batch_size=64,callbacks=[history])
result = model.evaluate(x_test,y_test)

for i in range(len(model.metrics_names)):
    print("Metric ",model.metrics_names[i],":",str(round(result[i],2)))

Epoch 1/15
11444/11444 [==============================] - 54s 4ms/step - loss: 1618486.0000 - mean_absolute_error: 837.5728 - val_loss: 1259849.8750 - val_mean_absolute_error: 738.6974
Epoch 2/15
11444/11444 [==============================] - 51s 4ms/step - loss: 1177575.6250 - mean_absolute_error: 726.6656 - val_loss: 1294894.6250 - val_mean_absolute_error: 732.8755
Epoch 3/15
11444/11444 [==============================] - 51s 4ms/step - loss: 1106218.0000 - mean_absolute_error: 703.5497 - val_loss: 1058466.0000 - val_mean_absolute_error: 693.3434
Epoch 4/15
11444/11444 [==============================] - 55s 5ms/step - loss: 1063446.5000 - mean_absolute_error: 689.9120 - val_loss: 1031878.9375 - val_mean_absolute_error: 688.3062
Epoch 5/15
11444/11444 [==============================] - 49s 4ms/step - loss: 1040647.3125 - mean_absolute_error: 683.1877 - val_loss: 1014320.7500 - val_mean_absolute_error: 676.1426
Epoch 6/15
11444/11444 [==============================] - 49s 4ms/step - lo

#Plotando a Métrica de Perda através das épocas
---

#Testando o Modelo
---
Prever manualmente a partir do modelo, em vez de usar a função de avaliação do modelo

In [58]:
y_test["Prediction"] = model.predict(x_test)
y_test.columns = ["Actual Sales","Predicted Sales"]
print(y_test.head(10))

6358/6358 [==============================] - 11s 2ms/step
        Actual Sales  Predicted Sales
115563             0         0.349251
832654             0         0.349251
769112          2933      3070.576904
350588          8602      7443.703125
141556          6975      6527.319336
84435           9239      8422.892578
53018              0         0.349251
262419             0         0.349251
702267          5885      5171.796387
981431             0         0.349251


In [59]:
#Manually predicting from the model, instead of using model's evaluate function
from sklearn.metrics import mean_squared_error, mean_absolute_error
print("MSE :",mean_squared_error(y_test["Actual Sales"].values,y_test["Predicted Sales"].values))
print("MAE :",mean_absolute_error(y_test["Actual Sales"].values,y_test["Predicted Sales"].values))

MSE : 876537.0571696477
MAE : 627.4551643114419


#CONCLUSÃO
---
- Começamos com a declaração do problema e a definimos usando estruturas padrão do setor para obter uma compreensão intuitiva de por que estamos resolvendo esse problema.
- Em seguida, exploramos os dados para compreender os recursos disponíveis e os diferentes tipos de dados

#FIM